# Inference in RAG

RAG - Retrieval-Augmented Generation (Generación aumentada con recuperación). Es una técnica dentro de la inteligencia artificial, sobre todo aplicada en modelos de lenguaje como los que se usan en chatbots o asistentes.

## Cómo funciona

	•	Recuperación (Retrieval): el sistema busca información relevante en una base de datos, documentos o fuentes externas.
    
	•	Generación (Generation): el modelo de lenguaje usa esa información recuperada para generar una respuesta más precisa, contextualizada y actualizada.

En otras palabras, el modelo no se limita solo a lo que ya aprendió durante su entrenamiento, sino que consulta información externa en tiempo real y la integra en su respuesta.

**Ejemplo práctico**

Si preguntas a un modelo con RAG: “¿Qué dice el informe de la ONU de 2024 sobre cambio climático?”, primero recuperará el texto del informe y luego generará una respuesta basada en ese contenido, en lugar de inventar o usar solo datos antiguos.

Ventajas:

	•	Respuestas más precisas y actualizadas.
    
	•	Reduce la alucinación (cuando un modelo inventa datos).
    
	•	Permite integrar bases de conocimiento privadas o especializadas (documentos internos, artículos científicos, manuales, etc.).


In [ ]:
#
import numpy as np

from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, \
                         DPRContextEncoder, DPRQuestionEncoder

import logging
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)

# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#
def cosine_similarity_matrix(features):
    norms = np.linalg.norm(features, axis=1, keepdims=True)
    normalized_features = features / norms
    similarity_matrix = np.inner(normalized_features, normalized_features)
    rounded_similarity_matrix = np.round(similarity_matrix, 4)
    return rounded_similarity_matrix

## Pure similarity

In [ ]:
#
answers = [
    "What is the tallest mountain in the world?",
    "The tallest mountain in the world is Mount Everest.",
    "Mount Shasta",
    "I like my hike in the mountains",
    "I am going to a yoga class"
]

question = 'What is the tallest mountain in the world?'

In [ ]:
#
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
question_embedding = list(model.encode(question))

In [ ]:
#
sim = []
for answer in answers:
    answer_embedding = list(model.encode(answer))
    sim.append(cosine_similarity_matrix(np.stack([question_embedding, answer_embedding]))[0,1])

print(sim)
best_inx = np.argmax(sim)
print(f"Question = {question}")
print(f"Best answer = {answers[best_inx]}")

## Dual-Encoder inference

In [ ]:
#
answer_tokenizer = AutoTokenizer \
                   .from_pretrained("facebook/dpr-ctx_encoder-multiset-base")
answer_encoder = DPRContextEncoder \
                   .from_pretrained("facebook/dpr-ctx_encoder-multiset-base")

question_tokenizer = AutoTokenizer \
                   .from_pretrained("facebook/dpr-question_encoder-multiset-base")
question_encoder = DPRQuestionEncoder \
                   .from_pretrained("facebook/dpr-question_encoder-multiset-base")

In [ ]:
# Compute the question embeddings
question_tokens = question_tokenizer(question, return_tensors="pt")["input_ids"]
question_embedding = question_encoder(question_tokens).pooler_output.flatten().tolist()


In [ ]:
#
print(question_embedding[:10], len(question_embedding))

In [ ]:
#
sim = []
for answer in answers:
    answer_tokens = answer_tokenizer(answer, return_tensors="pt")["input_ids"]
    answer_embedding = answer_encoder(answer_tokens).pooler_output.flatten().tolist()
    sim.append(cosine_similarity_matrix(np.stack([question_embedding, answer_embedding]))[0,1])


In [ ]:
#
print(sim)
best_inx = np.argmax(sim)
print(f"Question = {question}")
print(f"Best answer = {answers[best_inx]}")